In [1]:
import pandas as pd
import os

In [5]:
# Ottiene la directory di lavoro corrente
script_dir = os.getcwd()

# Costruisce il percorso relativo al file CSV
file_path = os.path.join(script_dir, "Data", "main_data", "releases.csv")

# Costruisce il percorso relativo al file CSV pulito
clean_path = os.path.join(script_dir, "Data", "clean_data")

# Crea la cartella clean_data se non esite
if not os.path.exists(clean_path):
    os.makedirs(clean_path)

# Carica il CSV in un DataFrame
df = pd.read_csv(file_path)

In [6]:
# Rimuove duplicati
df.drop_duplicates(inplace=True)

In [8]:
# Rimuove righe con troppi valori nulli (ad esempio, più del 50% dei dati mancanti)
df.dropna(thresh=df.shape[1] * 0.5, inplace=True)

# Sostituisce i valori nulli con "Unknown"
df['country'] = df['country'].fillna("Unknown")
df['date'] = df['date'].fillna("Unknown")
df['type'] = df['type'].fillna("Unknown")
df['rating'] = df['rating'].fillna("Unknown")

In [9]:
# Salva il file pulito
cleaned_csv_path = os.path.join(clean_path, "releases_cleaned.csv")
df.to_csv(cleaned_csv_path, index=False, encoding="utf-8")